## Preambule

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from pathlib import Path
from tqdm import tqdm
import xarray as xr

## Input

In [2]:
params = dict(start_year_analysis=2021)
path_alloc = "K:/ECEMF/T5.2/Allocations/"
path_save = "K:/ECEMF/T5.2/"

## Read

In [3]:
xr_dataread = xr.open_dataset(path_save + "xr_dataread.nc", engine='netcdf4')

In [4]:
#focusregs = ['EU', 'USA', 'IND', 'CHN', 'JPN', 'AUS', 'BRA', 'RUS', 'ZAF']
focusregs = np.array(xr_dataread.Region)
temperatures = [1.5, 2.0]
rules = ['GF', 'PC', 'PCC', 'ECPC', 'AP', 'GDR']

## Open datasets

Budgets

In [5]:
# for temp_i, temp in enumerate([1.5, 2.0, 2.5, 3.0]):
#     xrs = []
#     for cty_i, cty in tqdm(enumerate(focusregs)):
#         ds = xr.open_dataset("X:/user/dekkerm/Projects/ECEMF_T5.2/Data/Allocations/" + "xr_alloc_"+cty+".nc").sel(Temperature=temp, Risk=0.5, Time=np.arange(params['start_year_analysis'], 2101)).sum(dim='Time').expand_dims(Region=[cty])
#         for rule_i, rule in enumerate(rules):
#             if rule != 'ECPC':
#                 xr_i = xrt[rule].sum(dim='Time').expand_dims(Region=[cty])
#             else:
#                 xr_i = xrt[rule].expand_dims(Region=[cty])
#             xrs.append(xr_i)
#     xr.merge(xrs).astype("float32").to_netcdf("X:/user/dekkerm/Projects/ECEMF_T5.2/Data/xr_budgets_"+str(temp)[0]+str(temp)[2]+"_50.nc", format='NETCDF4')

Trajectory emissions

In [6]:
for temp_i, temp in enumerate([1.5]):
    for cty_i, cty in tqdm(enumerate(focusregs)):
        ds = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Temperature=temp, Risk=0.5, Time=np.arange(params['start_year_analysis'], 2101)).expand_dims(Region=[cty])
        if cty_i == 0:
            xrt = ds.copy()
        else:
            xrt = xr.merge([xrt, ds])
        ds.close()
    xrt.astype("float32").to_netcdf(path_save + "xr_traj_"+str(temp)[0]+str(temp)[2]+"_50.nc", format='NETCDF4')

# for temp_i, temp in enumerate(temperatures):
#     for cty_i, cty in enumerate(focusregs):
#         if cty_i == 0:
#             xrt = xr.open_dataset("K:/ECEMF/T5.2/Allocations/" + "xr_alloc_"+cty+".nc").sel(Temperature=temp, Risk=0.5, Time=np.arange(params['start_year_analysis'], 2101)).expand_dims(Region=[cty])
#         else:
#             try: xr_i = xr.open_dataset("K:/ECEMF/T5.2/Allocations/" + "xr_alloc_"+cty+".nc").sel(Temperature=temp, Risk=0.5, Time=np.arange(params['start_year_analysis'], 2101)).expand_dims(Region=[cty])
#             except: xr_i = xr.open_dataset("X:/user/dekkerm/Projects/ECEMF_T5.2/Data/Allocations/" + "xr_alloc_"+cty+".nc").sel(Temperature=temp, Risk=0.5, Time=np.arange(params['start_year_analysis'], 2101)).expand_dims(Region=[cty])
#             xrt = xr.merge([xrt, xr_i])
#     xrt.astype("float32").to_netcdf("X:/user/dekkerm/Projects/ECEMF_T5.2/Data/xr_traj_"+str(temp)[0]+str(temp)[2]+"_50.nc", format='NETCDF4')

0it [00:00, ?it/s]

207it [03:49,  1.11s/it]


2030 allocations

In [7]:
for cty_i, cty in tqdm(enumerate(focusregs)):
    ds = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Time=2030).expand_dims(Region=[cty])
    if cty_i == 0:
        xrt = ds.copy()
    else:
        xrt = xr.merge([xrt, ds])
    ds.close()
xrt.astype("float32").to_netcdf(path_save + "xr_alloc_2030.nc", format='NETCDF4')

207it [02:42,  1.27it/s]


2035 allocations

In [9]:
for cty_i, cty in tqdm(enumerate(focusregs)):
    ds = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Time=2035).expand_dims(Region=[cty])
    if cty_i == 0:
        xrt = ds.copy()
    else:
        xrt = xr.merge([xrt, ds])
    ds.close()
xrt.astype("float32").to_netcdf(path_save + "xr_alloc_2035.nc", format='NETCDF4')

0it [00:00, ?it/s]x:\user\dekkerm\environments\MyEnv2\lib\site-packages\xarray\core\alignment.py:307: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  index = joiner(matching_indexes)
2it [00:00, 19.64it/s]x:\user\dekkerm\environments\MyEnv2\lib\site-packages\xarray\core\alignment.py:307: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  index = joiner(matching_indexes)
x:\user\dekkerm\environments\MyEnv2\lib\site-packages\xarray\core\alignment.py:307: FutureWarning: Index.__or__ operating as a set operation is deprecated, in the future this will be a logical operation matching Series.__or__.  Use index.union(other) instead.
  index = joiner(matching_indexes)
4it [00:00, 17.03it/s]x:\user\dekkerm\environments\MyEnv2\lib\site-packages

2040 allocations

In [8]:
for cty_i, cty in tqdm(enumerate(focusregs)):
    ds = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Time=2040).expand_dims(Region=[cty])
    if cty_i == 0:
        xrt = ds.copy()
    else:
        xrt = xr.merge([xrt, ds])
    ds.close()
xrt.astype("float32").to_netcdf(path_save + "xr_alloc_2040.nc", format='NETCDF4')

0it [00:00, ?it/s]

207it [02:33,  1.35it/s]


2050 allocations

In [9]:
for cty_i, cty in tqdm(enumerate(focusregs)):
    if cty_i == 0:
        xrt = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Time=2050).expand_dims(Region=[cty])
    else:
        try: xr_i = xr.open_dataset("K:/ECEMF/T5.2/Allocations/" + "xr_alloc_"+cty+".nc").sel(Time=2050).expand_dims(Region=[cty])
        except: xr_i = xr.open_dataset("X:/user/dekkerm/Projects/ECEMF_T5.2/Data/Allocations/" + "xr_alloc_"+cty+".nc").sel(Time=2050).expand_dims(Region=[cty])
        xrt = xr.merge([xrt, xr_i])
xrt.astype("float32").to_netcdf(path_save + "xr_alloc_2050.nc", format='NETCDF4')

207it [02:29,  1.39it/s]


F_C budgets

In [10]:
for cty_i, cty in tqdm(enumerate(focusregs)):
    ds = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Time=np.arange(2021, 2101)).sum(dim='Time').expand_dims(Region=[cty])
    if cty_i == 0:
        xrt = ds.copy()
    else:
        xrt = xr.merge([xrt, ds])
    ds.close()
xrt.astype("float32").to_netcdf(path_save + "xr_alloc_FC.nc", format='NETCDF4')

207it [03:00,  1.15it/s]


For the uncertainty analysis

In [11]:
for temp_i, temp in enumerate(temperatures):
    xrs = []
    for cty_i, cty in tqdm(enumerate(focusregs)):
        #try: xrt = xr.open_dataset("K:/ECEMF/T5.2/Allocations/" + "xr_alloc_"+cty+".nc").sel(Temperature=temp, TrajUnc='Medium', NegEmis=0.5, Time=np.arange(params['start_year_analysis'], 2101))
        xrt = xr.open_dataset(path_alloc + "xr_alloc_"+cty+".nc").sel(Temperature=temp, TrajUnc='Medium', NegEmis=0.5, Time=np.arange(params['start_year_analysis'], 2101))
        for rule_i, rule in enumerate(rules):
            if rule != 'ECPC':
                xr_i = xrt[rule].sum(dim='Time').expand_dims(Region=[cty])
            else:
                xr_i = xrt[rule].expand_dims(Region=[cty])
            xrs.append(xr_i)
    xr.merge(xrs).astype("float32").to_netcdf(path_save + "xr_uncbudget_"+str(temp)[0]+str(temp)[2]+".nc", format='NETCDF4')

207it [00:07, 26.83it/s]
207it [00:07, 27.78it/s]
